# Análisis Exploratorio de Datos de Cáncer - TCIA

Este notebook proporciona un análisis exploratorio completo de datos de cáncer obtenidos de The Cancer Imaging Archive (TCIA). Incluye:

- Conexión y descarga de datos desde TCIA
- Procesamiento de imágenes DICOM
- Análisis estadístico descriptivo
- Visualizaciones interactivas
- Preprocesamiento para modelos de ML

## Configuración Inicial

In [ ]:
# Imports necesarios
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Agregar src al path
src_path = Path('../src').absolute()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Imports principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Configurar estilo de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print(f"Análisis iniciado: {datetime.now()}")
print(f"Directorio de trabajo: {os.getcwd()}")

In [ ]:
# Importar módulos del proyecto
try:
    from utils.tcia_client import TCIAClient
    from utils.dicom_processor import DICOMProcessor
    from utils.gemini_analyzer import GeminiAnalyzer
    print("✓ Módulos del proyecto importados exitosamente")
except ImportError as e:
    print(f"❌ Error importando módulos: {e}")
    print("Asegúrese de tener instaladas las dependencias requeridas")

## 1. Conexión con TCIA y Exploración de Colecciones

Primero exploraremos las colecciones disponibles en TCIA y seleccionaremos las más relevantes para análisis de cáncer.

In [ ]:
# Inicializar cliente TCIA
try:
    tcia = TCIAClient()
    print("✓ Cliente TCIA inicializado")
except Exception as e:
    print(f"❌ Error inicializando cliente TCIA: {e}")
    tcia = None

In [ ]:
# Obtener y analizar colecciones disponibles
if tcia:
    collections = tcia.get_collections()
    
    if collections:
        print(f"Total de colecciones disponibles: {len(collections)}")
        
        # Mostrar primeras colecciones
        collections_df = pd.DataFrame(collections)
        print("\nPrimeras 10 colecciones:")
        display(collections_df.head(10))
    else:
        print("No se pudieron obtener colecciones")
        collections_df = pd.DataFrame()

In [ ]:
# Analizar colecciones específicas de cáncer
target_collections = ['CMB-LCA', 'CMB-BRCA', 'CMB-CRC', 'CMB-MEL', 'AURORA-Metastatic-Breast-Multiomics']

collection_stats = []

for collection in target_collections:
    try:
        stats = tcia.get_collection_statistics(collection)
        if stats:
            collection_stats.append(stats)
            print(f"\n📊 Estadísticas para {collection}:")
            print(f"  - Pacientes: {stats['total_patients']}")
            print(f"  - Estudios: {stats['total_studies']}")
            print(f"  - Series: {stats['total_series']}")
            print(f"  - Modalidades: {stats['modalities']}")
    except Exception as e:
        print(f"❌ Error obteniendo estadísticas de {collection}: {e}")

In [ ]:
# Visualizar estadísticas de colecciones
if collection_stats:
    stats_df = pd.DataFrame(collection_stats)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Gráfico de pacientes por colección
    axes[0,0].bar(stats_df['collection'], stats_df['total_patients'])
    axes[0,0].set_title('Número de Pacientes por Colección')
    axes[0,0].set_xlabel('Colección')
    axes[0,0].set_ylabel('Número de Pacientes')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Gráfico de estudios por colección
    axes[0,1].bar(stats_df['collection'], stats_df['total_studies'])
    axes[0,1].set_title('Número de Estudios por Colección')
    axes[0,1].set_xlabel('Colección')
    axes[0,1].set_ylabel('Número de Estudios')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Gráfico de series por colección
    axes[1,0].bar(stats_df['collection'], stats_df['total_series'])
    axes[1,0].set_title('Número de Series por Colección')
    axes[1,0].set_xlabel('Colección')
    axes[1,0].set_ylabel('Número de Series')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Resumen estadístico
    axes[1,1].axis('off')
    summary_text = f"""
    Resumen de Colecciones:
    
    Total Pacientes: {stats_df['total_patients'].sum()}
    Total Estudios: {stats_df['total_studies'].sum()}
    Total Series: {stats_df['total_series'].sum()}
    
    Promedio por Colección:
    - Pacientes: {stats_df['total_patients'].mean():.1f}
    - Estudios: {stats_df['total_studies'].mean():.1f}
    - Series: {stats_df['total_series'].mean():.1f}
    """
    axes[1,1].text(0.1, 0.5, summary_text, fontsize=12, va='center')
    
    plt.tight_layout()
    plt.show()
    
    # Guardar estadísticas
    stats_df.to_csv('../results/collection_statistics.csv', index=False)
    print("\n💾 Estadísticas guardadas en results/collection_statistics.csv")

## 2. Descarga y Procesamiento de Muestra de Datos

Descargaremos una muestra pequeña de datos para análisis exploratorio.

In [ ]:
# Inicializar procesador DICOM
try:
    dicom_processor = DICOMProcessor()
    print("✓ Procesador DICOM inicializado")
except Exception as e:
    print(f"❌ Error inicializando procesador DICOM: {e}")
    dicom_processor = None

In [ ]:
# Descargar muestra pequeña de la colección de cáncer de pulmón
sample_collection = 'CMB-LCA'  # Lung Cancer

print(f"Descargando muestra de {sample_collection}...")
print("⚠️  Esto puede tomar algunos minutos...")

try:
    downloaded_files = tcia.download_collection_sample(
        collection=sample_collection,
        max_patients=2,  # Solo 2 pacientes para demo
        max_series_per_patient=1  # 1 serie por paciente
    )
    
    print(f"\n✓ Descargados {len(downloaded_files)} archivos:")
    for file in downloaded_files:
        print(f"  - {file}")
        
except Exception as e:
    print(f"❌ Error en descarga: {e}")
    downloaded_files = []

In [ ]:
# Procesar archivos DICOM descargados
if downloaded_files and dicom_processor:
    processing_results = []
    
    for i, zip_file in enumerate(downloaded_files[:2]):  # Procesar solo los primeros 2
        print(f"\nProcesando archivo {i+1}: {zip_file}")
        
        output_dir = f"../data/processed/sample_{i+1}"
        
        try:
            result = dicom_processor.process_dicom_series(zip_file, output_dir)
            processing_results.append(result)
            
            print(f"  ✓ Procesadas {len(result['processed_images'])} imágenes")
            if result['errors']:
                print(f"  ⚠️  {len(result['errors'])} errores encontrados")
                
        except Exception as e:
            print(f"  ❌ Error procesando {zip_file}: {e}")
    
    print(f"\n📊 Resumen del procesamiento:")
    total_images = sum(len(r['processed_images']) for r in processing_results)
    total_errors = sum(len(r['errors']) for r in processing_results)
    print(f"  - Total imágenes procesadas: {total_images}")
    print(f"  - Total errores: {total_errors}")
else:
    processing_results = []
    print("No hay archivos para procesar o procesador no disponible")

## 3. Análisis de Metadatos DICOM

Analizaremos los metadatos extraídos de las imágenes DICOM para entender mejor los datos.

In [ ]:
# Consolidar metadatos de todas las imágenes procesadas
all_metadata = []

for result in processing_results:
    all_metadata.extend(result['metadata'])

if all_metadata:
    metadata_df = pd.DataFrame(all_metadata)
    
    print(f"📋 Metadatos de {len(metadata_df)} imágenes:")
    print(f"Columnas disponibles: {list(metadata_df.columns)}")
    
    # Mostrar muestra de metadatos
    display(metadata_df.head())
    
    # Análisis de modalidades
    if 'Modality' in metadata_df.columns:
        modality_counts = metadata_df['Modality'].value_counts()
        print(f"\n🔬 Modalidades encontradas:")
        for modality, count in modality_counts.items():
            print(f"  - {modality}: {count} imágenes")
    
    # Análisis de dimensiones de imagen
    if 'Rows' in metadata_df.columns and 'Columns' in metadata_df.columns:
        print(f"\n📐 Dimensiones de imágenes:")
        print(f"  - Filas: {metadata_df['Rows'].describe()}")
        print(f"  - Columnas: {metadata_df['Columns'].describe()}")
        
        # Visualizar distribución de tamaños
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        metadata_df['Rows'].hist(bins=20, ax=axes[0])
        axes[0].set_title('Distribución de Filas')
        axes[0].set_xlabel('Número de Filas')
        
        metadata_df['Columns'].hist(bins=20, ax=axes[1])
        axes[1].set_title('Distribución de Columnas')
        axes[1].set_xlabel('Número de Columnas')
        
        plt.tight_layout()
        plt.show()
    
    # Guardar metadatos
    metadata_df.to_csv('../results/dicom_metadata.csv', index=False)
    print("\n💾 Metadatos guardados en results/dicom_metadata.csv")
    
else:
    print("No hay metadatos para analizar")
    metadata_df = pd.DataFrame()

## 4. Visualización de Imágenes Procesadas

Visualizaremos algunas de las imágenes procesadas para verificar la calidad del preprocesamiento.

In [ ]:
# Encontrar imágenes procesadas
processed_images = []

for result in processing_results:
    processed_images.extend(result['processed_images'])

print(f"🖼️  Imágenes procesadas encontradas: {len(processed_images)}")

if processed_images:
    # Mostrar algunas imágenes
    n_display = min(6, len(processed_images))
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i in range(n_display):
        try:
            img_path = processed_images[i]
            img = plt.imread(img_path)
            
            axes[i].imshow(img, cmap='gray')
            axes[i].set_title(f'Imagen {i+1}\n{Path(img_path).name}')
            axes[i].axis('off')
            
            # Mostrar estadísticas básicas
            axes[i].text(0.02, 0.98, f'Shape: {img.shape}\nMin: {img.min():.2f}\nMax: {img.max():.2f}', 
                        transform=axes[i].transAxes, va='top', ha='left',
                        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            
        except Exception as e:
            axes[i].text(0.5, 0.5, f'Error cargando\nimagen {i+1}\n{e}', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')
    
    # Ocultar axes vacíos
    for i in range(n_display, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle('Muestra de Imágenes Procesadas', fontsize=16)
    plt.tight_layout()
    plt.show()
    
else:
    print("No hay imágenes procesadas para mostrar")

## 5. Análisis con Gemini AI (Opcional)

Si la API de Gemini está configurada, realizaremos análisis de algunas imágenes.

In [ ]:
# Inicializar analizador Gemini (opcional)
try:
    gemini_analyzer = GeminiAnalyzer()
    print("✓ Analizador Gemini inicializado")
    gemini_available = True
except Exception as e:
    print(f"⚠️  Analizador Gemini no disponible: {e}")
    print("   Verifique que tenga configurada la API key de Gemini")
    gemini_available = False

In [ ]:
# Análisis con Gemini de algunas imágenes
if gemini_available and processed_images:
    print("🤖 Analizando imágenes con Gemini AI...")
    
    # Analizar las primeras 2 imágenes
    gemini_results = []
    
    for i, img_path in enumerate(processed_images[:2]):
        print(f"\nAnalizando imagen {i+1}: {Path(img_path).name}")
        
        try:
            result = gemini_analyzer.analyze_medical_image(
                img_path, 
                analysis_type="cancer_detection"
            )
            gemini_results.append(result)
            
            print(f"✓ Análisis completado")
            
            # Mostrar resumen del análisis
            if 'gemini_response' in result:
                response = result['gemini_response'][:200] + "..." if len(result['gemini_response']) > 200 else result['gemini_response']
                print(f"  Respuesta: {response}")
            
            if 'confidence_indicators' in result and result['confidence_indicators']:
                print(f"  Indicadores de confianza: {result['confidence_indicators']}")
                
        except Exception as e:
            print(f"❌ Error analizando imagen {i+1}: {e}")
    
    # Guardar resultados de Gemini
    if gemini_results:
        with open('../results/gemini_analysis.json', 'w') as f:
            json.dump(gemini_results, f, indent=2)
        print("\n💾 Resultados de Gemini guardados en results/gemini_analysis.json")
        
else:
    print("🔒 Análisis con Gemini no disponible o no hay imágenes")
    gemini_results = []

## 6. Resumen y Conclusiones

Resumen del análisis exploratorio realizado.

In [ ]:
# Generar resumen final
print("📊 RESUMEN DEL ANÁLISIS EXPLORATORIO")
print("=" * 50)

print(f"\n🗂️  DATOS ANALIZADOS:")
print(f"  - Colecciones TCIA exploradas: {len(target_collections)}")
print(f"  - Archivos DICOM descargados: {len(downloaded_files)}")
print(f"  - Imágenes procesadas: {len(processed_images)}")
print(f"  - Metadatos extraídos: {len(all_metadata)}")

if collection_stats:
    total_patients = sum(s['total_patients'] for s in collection_stats)
    total_studies = sum(s['total_studies'] for s in collection_stats)
    total_series = sum(s['total_series'] for s in collection_stats)
    
    print(f"\n📈 ESTADÍSTICAS DE COLECCIONES:")
    print(f"  - Total pacientes disponibles: {total_patients}")
    print(f"  - Total estudios disponibles: {total_studies}")
    print(f"  - Total series disponibles: {total_series}")

if not metadata_df.empty and 'Modality' in metadata_df.columns:
    print(f"\n🔬 MODALIDADES PROCESADAS:")
    for modality, count in metadata_df['Modality'].value_counts().items():
        print(f"  - {modality}: {count} imágenes")

if gemini_results:
    print(f"\n🤖 ANÁLISIS CON IA:")
    print(f"  - Imágenes analizadas con Gemini: {len(gemini_results)}")
    print(f"  - Resultados guardados en: results/gemini_analysis.json")

print(f"\n💾 ARCHIVOS GENERADOS:")
output_files = [
    'results/collection_statistics.csv',
    'results/dicom_metadata.csv'
]
if gemini_results:
    output_files.append('results/gemini_analysis.json')

for file in output_files:
    if Path(f'../{file}').exists():
        print(f"  ✓ {file}")
    else:
        print(f"  ❌ {file} (no generado)")

print(f"\n🎯 PRÓXIMOS PASOS:")
print(f"  1. Descargar conjuntos de datos más grandes")
print(f"  2. Realizar análisis radiómico detallado")
print(f"  3. Entrenar modelos de deep learning")
print(f"  4. Validar modelos con datos independientes")
print(f"  5. Desarrollar pipeline de producción")

print(f"\n✅ Análisis exploratorio completado: {datetime.now()}")